In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor



In [10]:
training_data = datasets.FashionMNIST(
            root="data",
            train=True,
            download=True,
            transform=ToTensor(),
        )

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [11]:
batch_size = 64
# create data loader


train_dataloader = DataLoader(training_data, batch_size)
test_dataloader = DataLoader(test_data, batch_size)

for X, y in train_dataloader:
    print(f"Shape of X [batch size, channels, height in pixels, width in pixels]: {X.shape}")
    print(f"Shape of y: {y.shape}, {y.dtype}")
    break

for X, y in test_dataloader:
    print(f"Shape of X [batch size, channels, height in pixels, width in pixels]: {X.shape}")
    print(f"Shape of y: {y.shape}, {y.dtype}")
    break


Shape of X [batch size, channels, height in pixels, width in pixels]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]), torch.int64
Shape of X [batch size, channels, height in pixels, width in pixels]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]), torch.int64


In [12]:
# get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"current using device {device}")

# define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        # flatten is used to convert [batch_size, channel, height, width] to [batch_size, channel * height * width]
        self.flatten = nn.Flatten()

        """n PyTorch, nn.Sequential is a container module that allows you to stack layers in a sequential
        manner. It takes in a sequence of layers as input and applies them in order to
        the input data when called."""
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features= 28 * 28, out_features= 512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)


current using device cpu
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
# define loss function
loss_fn = nn.CrossEntropyLoss()
# define 优化器
optimizer = torch.optim.SGD(model.parameters() , lr = 1e-3)

In [14]:
# so now we define the traning function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # turn the training model on
    for batch, (X,y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(batch % 100 == 0):
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")





In [15]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305907  [   64/60000]
loss: 2.294276  [ 6464/60000]
loss: 2.266275  [12864/60000]
loss: 2.260387  [19264/60000]
loss: 2.251948  [25664/60000]
loss: 2.214518  [32064/60000]
loss: 2.231687  [38464/60000]
loss: 2.189763  [44864/60000]
loss: 2.177109  [51264/60000]
loss: 2.166358  [57664/60000]
Test Error: 
 Accuracy: 38.2%, Avg loss: 2.147210 

Epoch 2
-------------------------------
loss: 2.156592  [   64/60000]
loss: 2.153106  [ 6464/60000]
loss: 2.081527  [12864/60000]
loss: 2.102928  [19264/60000]
loss: 2.059895  [25664/60000]
loss: 1.992649  [32064/60000]
loss: 2.032161  [38464/60000]
loss: 1.940350  [44864/60000]
loss: 1.938021  [51264/60000]
loss: 1.898036  [57664/60000]
Test Error: 
 Accuracy: 55.0%, Avg loss: 1.875042 

Epoch 3
-------------------------------
loss: 1.902006  [   64/60000]
loss: 1.885900  [ 6464/60000]
loss: 1.748400  [12864/60000]
loss: 1.803808  [19264/60000]
loss: 1.697215  [25664/60000]
loss: 1.644708  [32064/600

In [16]:
# save model
torch.save(model.state_dict() , "model.pth")
print("saved the pytorch model state to model.pth")

saved the pytorch model state to model.pth


In [17]:
# load model
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>